## Tips, Tricks, Traps and good manners
Basic examples of tips, and things to do and avoid

In [ ]:
import torch

import laueimproc

### Traps
This section shows several traps and how to avoid falling into them.

#### Bounding boxes to surface overflow

In [ ]:
# init
diagram = laueimproc.Diagram(laueimproc.io.get_sample())
diagram.set_spots([
    [0, 0, 10, 10],  # little bbox
    [0, 0, 255, 255],  # big bbox
    [0, 0, 256, 256],  # very big bbox
])

In [ ]:
# mistake to avoid
areas = diagram.bboxes[:, 2] * diagram.bboxes[:, 3]  # surface = height * width
print(areas)

In [ ]:
# solution
areas = diagram.areas
print(areas)

#### Shared references, inplace modifications

In [ ]:
# init
dataset = laueimproc.DiagramsDataset(laueimproc.io.get_samples())
prop = ["prop1"]
dataset.add_property("prop", prop, erasable=False)

print(dataset.get_property("prop"))
print(dataset[0].state)

In [ ]:
# clone trap
dataset_view = dataset[:10]
dataset_view.get_property("prop").append("I am a pirate!")  # change property state
dataset_view[0].find_spots()  # change diagram state

print(dataset.get_property("prop"))
print(dataset[0].state)

In [ ]:
# solution (run again init cell)
dataset_view = dataset[:10].clone()
dataset_view.get_property("prop").append("I am a pirate!")  # change property state
dataset_view[0].find_spots()  # change diagram state

print(dataset.get_property("prop"))
print(dataset[0].state)

### Tips
* shorcut to simplify you life!

#### Numpy convention (i, j, origin upper left (0, 0)) to LaueTools convention (x, y, origin center (1, 1))

In [ ]:
# init
diagram = laueimproc.Diagram(laueimproc.io.get_sample())
diagram.find_spots()

In [ ]:
# non elegance to avoid
positions = diagram.compute_rois_centroid()  # numpy conv ij
positions = positions.flip(dims=[1]) + 0.5  # it is slow, eavy and source of mistakes

In [ ]:
# best solution
positions = diagram.compute_rois_centroid(conv="xy")

In [ ]:
# if the argument conv is not provided
from laueimproc.convention import ij_to_xy
positions = diagram.compute_rois_centroid(conv="ij")  # let image it comes from somewere else
positions = ij_to_xy(positions, i=(..., 0), j=(..., 1))

### Good Manners
* To make your code faster
* To reduce the amongt of memory used

#### Don't cache when you don't need it after

In [ ]:
# init
diagram = laueimproc.Diagram(laueimproc.io.get_sample())
diagram.find_spots()

In [ ]:
positions = diagram.compute_rois_centroid(cache=False)

#### Convention ij faster than xy, please use ij conv as mush as possible

In [ ]:
# init
image_ij = torch.randn((2048, 2048))
image_xy = (image_ij + 0.5).mT

In [ ]:
# timer
import timeit
time_ij = min(timeit.repeat(image_ij.clone, number=100, repeat=10))
time_xy = min(timeit.repeat(image_xy.clone, number=100, repeat=10))
print(f"the copy ij is {time_xy/time_ij:.2f} time faster than the copy xy")